In [1]:
import mlflow
import optuna

import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split,cross_val_predict,StratifiedKFold
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tracking_uri = "http://ec2-16-170-247-92.eu-north-1.compute.amazonaws.com:5000" 

In [3]:
df  = pd.read_csv("data/preprocessing_df.csv")
df

,Unnamed: 0,text,category,word_count,num_chars,num_punctuation_chars
0,0,family mormon never tried explain still stare ...,1,39,259,0
1,1,buddhism much lot compatible christianity espe...,1,196,1268,0
2,2,seriously say thing first get complex explain ...,-1,86,459,0
3,3,learned want teach different focus goal wrappi...,0,29,167,0
4,4,benefit may want read living buddha living chr...,1,112,690,0
...,...,...,...,...,...,...
199697,162975,crore paid neerav modi recovered congress lead...,-1,18,107,0
199698,162976,dear rss terrorist payal gawar modi killing pl...,-1,36,248,0
199699,162977,cover interaction forum left,0,9,50,0
199700,162978,big project came india modi dream project happ...,0,13,77,0


In [4]:
df.text = df.text.fillna("")
df.category = df.category.map({-1:2,0:0,1:1})
X = df.text
y = df.category
ngram_range = (1,3)
max_features = 8000
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=114, stratify=y)

vectorizer = TfidfVectorizer(max_features=max_features,ngram_range=ngram_range)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

smote = SMOTE(random_state=114)
X_train_vec,y_train = smote.fit_resample(X_train_vec,y_train)

In [5]:
mlflow.set_tracking_uri(tracking_uri)
mlflow.set_experiment("Exp5 : ML Algorithms with HiperParameter Tuning")

2025/06/28 18:53:11 INFO mlflow.tracking.fluent: Experiment with name 'Exp5 : ML Algorithms with HiperParameter Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://mlflow-bucket-114/247004983170144617', creation_time=1751125991263, experiment_id='247004983170144617', last_update_time=1751125991263, lifecycle_stage='active', name='Exp5 : ML Algorithms with HiperParameter Tuning', tags={}>

In [6]:
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

In [7]:
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train_vec, y_train).predict(X_test_vec))


In [9]:
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the experiment for XGBoost
run_optuna_experiment()

[I 2025-06-28 18:54:27,183] A new study created in memory with name: no-name-89e69899-f63f-4b9a-a1bb-b917c5968127
[I 2025-06-28 18:56:11,440] Trial 0 finished with value: 0.6036654064745499 and parameters: {'n_estimators': 264, 'learning_rate': 0.009157341669041605, 'max_depth': 4}. Best is trial 0 with value: 0.6036654064745499.
[I 2025-06-28 19:10:25,405] Trial 1 finished with value: 0.653989634711199 and parameters: {'n_estimators': 162, 'learning_rate': 0.011739999766078488, 'max_depth': 9}. Best is trial 1 with value: 0.653989634711199.
[W 2025-06-28 19:12:52,942] Trial 2 failed with parameters: {'n_estimators': 168, 'learning_rate': 0.02785379425730787, 'max_depth': 9} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/j2/5thpfhw90bnddzd8vf1y20dr0000gn

KeyboardInterrupt: 